### Access A2AJ's Canadian legal data programmatically via API

Documentation available at: https://api.a2aj.ca/docs

NOTE: The API is meant to faciliate programmatic access for searching cases and legislation/regulations, and for obtaining the full text of a small number of cases or legislation/regulations. To access bulk data, instead download the Hugging Face dataset (access-via-hugging-face.ipynb) or download parquet files (access-via-parquet.ipynb). 


#### Get case coverage

Returns statistics on courts and tribunals covered in the dataset.

In [1]:
import requests
import pandas as pd

# set url
url = "https://api.a2aj.ca/coverage"

# get parameters
doc_type = "cases" 
parameters = f"doc_type={doc_type}"

# get data
request = requests.get(f"{url}?{parameters}")
data = request.json()

# convert to dataframe
df = pd.DataFrame(data['results'])
df

,dataset,description_en,description_fr,earliest_document_date,latest_document_date,number_of_documents
0,CHRT,Canadian Human Rights Tribunal,Tribunal canadien des droits de la personne,2003-01-10T00:00:00,2025-07-07T00:00:00,1049
1,CMAC,Court Martial Appeal Court,Cour d’appel de la cour martiale,2001-01-19T00:00:00,2025-06-17T00:00:00,147
2,FC,Federal Court,Cour fédérale,2001-02-01T00:00:00,2025-07-28T00:00:00,34245
3,FCA,Federal Court of Appeal,Cour d’appel fédérale,2001-02-01T00:00:00,2025-07-25T00:00:00,7579
4,ONCA,Ontario Court of Appeal,Cour d’appel de l’Ontario,2007-01-02T00:00:00,2025-07-28T00:00:00,16937
5,RAD,Refugee Appeal Division (IRB),Section d’appel des réfugiés (CIDR),2013-02-19T00:00:00,2024-07-22T00:00:00,14022
6,RLLR,"Refugee Law Lab Reporter (RPD, IRB)",None,2019-01-07T00:00:00,2023-12-29T00:00:00,898
7,RPD,Refugee Protection Division (IRB),Section de la protection des réfugiés (CIDR),2002-07-16T00:00:00,2020-12-14T00:00:00,6729
8,SCC,Supreme Court of Canada,Cour suprême du Canada,1877-01-15T00:00:00,2025-07-25T00:00:00,10844
9,SST,Social Security Tribunal,Tribunal de la sécurité sociale,2013-03-08T00:00:00,2025-12-16T00:00:00,16304


#### Get case by citation

Returns the full text of cases by citation.

In [2]:
import requests
import pandas as pd

# set API URL
url = "https://api.a2aj.ca/case-by-citation"

# get parameters
citation = "2023 SCC 17"  # required
output_language = "en"  # optional, "en", "fr" or "both", default is "en"
start_char = "0" # optional for chunking, default is 0
end_char = "-1" # optional for chunking, default is -1 (i.e. to end of text)
parameter_list = [f"citation={citation}",
                  f"output_language={output_language}", 
                  f"start_char={start_char}", 
                  f"end_char={end_char}"]
parameters = "&".join(parameter_list)

# get data
request = requests.get(f"{url}?{parameters}")
data = request.json()

# convert to dataframe
df = pd.DataFrame(data["results"])
df


,dataset,citation_en,citation2_en,name_en,document_date_en,url_en,scraped_timestamp_en,unofficial_text_en,upstream_license
0,SCC,2023 SCC 17,,Canadian Council for Refugees v. Canada (Citiz...,2023-06-16T00:00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2023-12-01T10:37:13.090000,Canadian Council for Refugees v. Canada (Citiz...,"See upstream license, including non-commercial..."


#### Get case text search

Search the full text of cases, returning case names, citations, snippets, etc.

NOTE: Supports advanced operators. - Boolean (AND, OR, NOT) or (ET, OU, NON) - Phrases: "Smith v. Jones" - Grouping: (Smith OR Jones) AND Roe - Wildcards: Smith* - Proximity: "Smith Jones"~3 or Smith NEAR/3 Jones or Smith /3 Jones

In [3]:
import requests
import pandas as pd

# set API URL
url = "https://api.a2aj.ca/case-text-search"

# get parameters
query = "right to housing"  # required, can include advanced operators described above
size = "10"  # optional, max is 50
search_language = "en"  # optional, "en" or "fr", default is "en"
sort_results = "default"  # optional, "default", "newest_first", "oldest_first" (default is relevance / boosting)
dataset = "CHRT, SCC, ONCA"  # optional, comma-separated list of datasets to search (default is all)
start_date = "2020-01-01" # optional yyyy-mm-dd, default is none
end_date = "2025-06-01" # optional yyyy-mm-dd , default none
list_of_parameters = [f"query={query}",
                      f"size={size}",
                      f"search_language={search_language}",
                      f"sort_results={sort_results}",
                      f"dataset={dataset}",
                      f"start_date={start_date}",
                      f"end_date={end_date}"]
parameters = "&".join(list_of_parameters)

# get data
request = requests.get(f"{url}?{parameters}")
data = request.json()

# convert to dataframe
df = pd.DataFrame(data["results"])
df

,dataset,citation_en,citation2_en,name_en,document_date_en,url_en,scraped_timestamp_en,citation_fr,citation2_fr,name_fr,document_date_fr,url_fr,scraped_timestamp_fr,score,snippet
0,SCC,2022 SCC 6,[2022] 1 SCR 29,Anderson v. Alberta,2022-03-18T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2025-01-03T11:24:10.508000+00:00,2022 CSC 6,[2022] 1 RCS 29,Anderson c. Alberta,2022-03-18T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2025-01-03T11:24:16.992000+00:00,701.01166,"Here, for example, it was unclear just how muc..."
1,SCC,2021 SCC 30,[2021] 2 SCR 571,Canada v. Canada North Group Inc.,2021-07-28T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2024-07-12T17:51:01.222000+00:00,2021 CSC 30,[2021] 2 RCS 571,Canada c. Canada North Group Inc.,2021-07-28T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2024-07-12T17:51:08.864000+00:00,516.64703,. 227(4.1) includes a <em>right</em> not <em>t...
2,SCC,2024 SCC 12,,Shot Both Sides v. Canada,2024-04-12T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2024-07-12T17:54:14.766000+00:00,2024 CSC 12,,Shot Both Sides c. Canada,2024-04-12T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2024-07-12T17:54:21.391000+00:00,435.87302,Prior <em>to</em> the coming into force of s. ...
3,SCC,2024 SCC 17,,St. John’s (City) v. Lynch,2024-05-10T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2024-07-12T17:48:34.128000+00:00,2024 CSC 17,,St. John’s (Ville) c. Lynch,2024-05-10T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2024-07-12T17:48:29.189000+00:00,423.99976,they have a <em>right</em> <em>to</em> proceed...
4,ONCA,2025 ONCA 78,,Windsor Housing Providers Inc. v. Windsor (City),2025-02-03T00:00:00+00:00,https://coadecisions.ontariocourts.ca/coa/coa/...,2025-05-31T06:27:03.174000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,418.63740,"The definitions of “Operate”, “Operator” and “..."
5,ONCA,2022 ONCA 313,,Smith v. Youthlink Youth Services,2022-04-25T00:00:00+00:00,https://coadecisions.ontariocourts.ca/coa/coa/...,2025-05-31T11:12:30.892000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,410.18286,The program also provides support in helping y...
6,ONCA,2024 ONCA 183,,Heinekamp (Re),2024-03-14T00:00:00+00:00,https://coadecisions.ontariocourts.ca/coa/coa/...,2025-05-31T01:52:36.275000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,405.52620,"The Board concluded that, while it was open <e..."
7,SCC,2024 SCC 24,,Iris Technologies Inc. v. Canada,2024-06-28T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2024-07-12T17:43:21.453000+00:00,2024 CSC 24,,Iris Technologies Inc. c. Canada,2024-06-28T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2024-07-12T17:43:27.693000+00:00,379.59375,rise <em>to</em> a <em>right</em> of appeal <e...
8,SCC,2021 SCC 34,[2021] 2 SCR 845,Toronto (City) v. Ontario (Attorney General),2021-10-01T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2024-11-01T21:42:52.766000+00:00,2021 CSC 34,[2021] 2 RCS 845,Toronto (Cité) c. Ontario (Procureur général),2021-10-01T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/f...,2024-11-01T21:42:46.303000+00:00,379.34387,governmental action in order <em>to</em> get a...
9,ONCA,2020 ONCA 651,,Shortt (Re),2020-10-16T00:00:00+00:00,https://coadecisions.ontariocourts.ca/coa/coa/...,2025-05-31T09:18:55.662000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,375.34402,Shortt was asking us <em>to</em> order “the St...


### Get case name search

Search names of cases (i.e. parties), and returns full case names, citations, etc.

NOTE: Supports advanced operators: AND/ET, OR/OU, NOT/NON, phrases in quotes, wildcards (*), proximity ("Smith Jones"~3 or Smith NEAR/3 Jones). Search targets only name_en or name_fr depending on language.

In [4]:
import requests
import pandas as pd

# set API URL
url = "https://api.a2aj.ca/case-name-search"

# get parameters
query = "roncarelli"  # required, can include advanced operators described above
size = "10"  # optional, max is 50
search_language = "en"  # optional, "en" or "fr", default is "en"
sort_results = "default"  # optional, "default", "newest_first", "oldest_first" (default is relevance / boosting)
dataset = "SCC"  # optional, comma-separated list of datasets to search (default is all)
start_date = "1900-01-01" # optional yyyy-mm-dd, default is none
end_date = "2000-12-31" # optional yyyy-mm-dd , default none
list_of_parameters = [f"query={query}",
                      f"size={size}",
                      f"search_language={search_language}",
                      f"sort_results={sort_results}",
                      f"dataset={dataset}",
                      f"start_date={start_date}",
                      f"end_date={end_date}"]
parameters = "&".join(list_of_parameters)

# get data
request = requests.get(f"{url}?{parameters}")
data = request.json()

# convert to dataframe
df = pd.DataFrame(data["results"])
df

,dataset,citation_en,citation2_en,name_en,document_date_en,url_en,scraped_timestamp_en,score
0,SCC,[1959] SCR 121,[1959] SCR 121,Roncarelli v. Duplessis,1959-01-27T00:00:00+00:00,https://decisions.scc-csc.ca/scc-csc/scc-csc/e...,2022-08-31T18:00:54.549000+00:00,0.329278


### Get laws coverage

Returns statistics on legislation and regulations covered in the dataset.

In [5]:
import requests
import pandas as pd

# set API URL
url = "https://api.a2aj.ca/coverage"

# get parameters
doc_type = "laws" 
parameters = f"doc_type={doc_type}"

# get data
request = requests.get(f"{url}?{parameters}")
data = request.json()

# convert to dataframe
df = pd.DataFrame(data['results'])
df
        

,dataset,description_en,description_fr,earliest_document_date,latest_document_date,number_of_documents
0,LEGISLATION-FED,Federal Statutes,Lois fédérales,1870-05-12T00:00:00,2025-06-26T00:00:00,954
1,REGULATIONS-FED,Federal Regulations,Règlements fédéraux,1945-12-21T00:00:00,2025-07-16T00:00:00,4803


#### Get law by citation

Returns the full text of legislation or regulations (or specific sections) by citation.

In [6]:
import requests
import pandas as pd

# set API URL
url = "https://api.a2aj.ca/law-by-citation"

# get parameters
citation = "RSC 1985, c A-1"  # required
output_language = "en"  # optional, "en", "fr" or "both", default is "en"
section = ""  # optional, section to retrieve, default is all sections
start_char = "0" # optional for chunking, default is 0
end_char = "-1" # optional for chunking, default is -1 (i.e. to end of text)
list_of_parameters = [f"citation={citation}",
                      f"output_language={output_language}",
                      f"start_char={start_char}",
                      f"end_char={end_char}"]
parameters = "&".join(list_of_parameters)

# get data
request = requests.get(f"{url}?{parameters}")
data = request.json()

# convert to dataframe
df = pd.DataFrame(data["results"])
df


,citation_en,citation2_en,dataset,name_en,document_date_en,source_url_en,scraped_timestamp_en,unofficial_text_en,upstream_license
0,"RSC 1985, c A-1",A-1,LEGISLATION-FED,Access to Information Act,1988-12-12T00:00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-29T00:00:00,"# Access to Information Act\n\nRSC 1985, c A-1...","See upstream license, including requirements r..."


### Get law text search

Search the full text of legislation/regulations, returning legislation/regulation names, citations, snippets, etc.

NOTE: Supports advanced operators. - Boolean (AND, OR, NOT) or (ET, OU, NON) - Phrases: "Smith v. Jones" - Grouping: (Smith OR Jones) AND Roe - Wildcards: Smith* - Proximity: "Smith Jones"~3 or Smith NEAR/3 Jones or Smith /3 Jones

In [7]:
import requests
import pandas as pd

# set API URL
url = "https://api.a2aj.ca/law-text-search"

# get parameters
query = "elephant"  # required, can include advanced operators described above
size = "10"  # optional, max is 50
search_language = "en"  # optional, "en" or "fr", default is "en"
sort_results = "default"  # optional, "default", "newest_first", "oldest_first" (default is relevance / boosting)
dataset = ""  # optional, comma-separated list of datasets to search (default is all)
start_date = "" # optional yyyy-mm-dd, default is none
end_date = "" # optional yyyy-mm-dd , default none
list_of_parameters = [f"query={query}",
                      f"size={size}",
                      f"search_language={search_language}",
                      f"sort_results={sort_results}",
                      f"dataset={dataset}",
                      f"start_date={start_date}",
                      f"end_date={end_date}"]
parameters = "&".join(list_of_parameters)

# get data
request = requests.get(f"{url}?{parameters}")
data = request.json()

# convert to dataframe
df = pd.DataFrame(data["results"])
df

,citation_en,citation2_en,dataset,name_en,document_date_en,source_url_en,scraped_timestamp_en,num_sections_en,citation_fr,citation2_fr,name_fr,document_date_fr,source_url_fr,scraped_timestamp_fr,num_sections_fr,score,snippet
0,SOR/96-263,,REGULATIONS-FED,Wild Animal and Plant Trade Regulations,1996-05-14T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,23,DORS/96-263,,Règlement sur le commerce d’espèces animales e...,1996-05-14T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,23,98.55667,. \n\n## <em>Elephant</em> Tusk and Rhinocero...


#### Get law name search

Search names of legislation/regulations, and returns full names, citations, etc.

NOTE: Supports advanced operators: AND/ET, OR/OU, NOT/NON, phrases in quotes, wildcards (*), proximity ("Smith Jones"~3 or Smith NEAR/3 Jones). Search targets only name_en or name_fr depending on language.

In [8]:
import requests
import pandas as pd

# set API URL
url = "https://api.a2aj.ca/law-name-search"

# get parameters
query = "privacy"  # required, can include advanced operators described above
size = "10"  # optional, max is 50
search_language = "en"  # optional, "en" or "fr", default is "en"
sort_results = "default"  # optional, "default", "newest_first", "oldest_first" (default is relevance / boosting)
dataset = ""  # optional, comma-separated list of datasets to search (default is all)
start_date = "" # optional yyyy-mm-dd, default is none
end_date = "" # optional yyyy-mm-dd , default none
list_of_parameters = [f"query={query}",
                      f"size={size}",
                      f"search_language={search_language}",
                      f"sort_results={sort_results}",
                      f"dataset={dataset}",
                      f"start_date={start_date}",
                      f"end_date={end_date}"]
parameters = "&".join(list_of_parameters)

# get data
request = requests.get(f"{url}?{parameters}")
data = request.json()

# convert to dataframe
df = pd.DataFrame(data["results"])
df

,citation_en,citation2_en,dataset,name_en,document_date_en,source_url_en,scraped_timestamp_en,num_sections_en,citation_fr,citation2_fr,name_fr,document_date_fr,source_url_fr,scraped_timestamp_fr,num_sections_fr,score
0,SI/2024-62,,REGULATIONS-FED,Privacy Act Heads of Government Institutions D...,2024-12-18T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,3,TR/2024-62,,Décret sur la désignation des responsables d’i...,2024-12-18T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,3,103.322490
1,SOR/2021-174,,REGULATIONS-FED,"Privacy Act Extension Order, No. 3",2021-07-14T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,2,DORS/2021-174,,Décret d’extension no 3 (Loi sur la protection...,2021-07-14T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,2,87.721120
2,"RSC 1985, c P-21",P-21,LEGISLATION-FED,Privacy Act,1988-12-12T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,91,"LRC 1985, c P-21",P-21,Loi sur la protection des renseignements perso...,1988-12-12T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,91,74.762680
3,SOR/83-508,,REGULATIONS-FED,Privacy Regulations,1983-06-03T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,14,DORS/83-508,,Règlement sur la protection des renseignements...,1983-06-03T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,14,9.841216
4,SOR/89-206,,REGULATIONS-FED,"Privacy Act Extension Order, No. 2",1989-04-13T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,2,DORS/89-206,,Décret d’extension no 2 (Loi sur la protection...,1989-04-13T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,2,7.528902
5,SOR/83-553,,REGULATIONS-FED,Privacy Act Extension Order No. 1,1983-06-24T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,2,DORS/83-553,,Décret d’extension no 1 (Loi sur la protection...,1983-06-24T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,2,3.189959
6,"CRC, c 440",,REGULATIONS-FED,Protection of Privacy Regulations,1979-08-15T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,4,"CRC, ch 440",,Règlement sur la protection de la vie privée,1979-08-15T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,4,2.747613
7,SI/83-114,,REGULATIONS-FED,Privacy Act Heads of Government Institutions D...,1983-07-13T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,0,TR/83-114,,Décret sur la désignation des responsables d’i...,1983-07-13T00:00:00+00:00,https://github.com/justicecanada/laws-lois-xml...,2025-07-27T00:00:00+00:00,0,0.000000
